<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Файл-сервера-xmlrpc_server_p3.ipynb-на-Python-3" data-toc-modified-id="Файл-сервера-xmlrpc_server_p3.ipynb-на-Python-3-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Файл сервера xmlrpc_server_p3.ipynb на Python 3</a></span></li></ul></div>

#### Файл сервера xmlrpc_server_p3.ipynb на Python 3 

In [ ]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import socket

import xmlrpc.client

import datetime
import pickle
import pandas as pd


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

server.register_introspection_functions()

stats_server = xmlrpc.client.ServerProxy("http://localhost:8018")


# Добавление в лог через сервер
# def add_log(log_line):
#     print('Add log')
#     try:
#         stats_server.add_log(log_line)
#     except Exception as e:
#         print('Stats server error!')
#     return True
    

# Тест
def ping():
#     add_log('ping')
#     print('exc pr')
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
#     add_log('now')    
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
#     add_log('type')
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
#     add_log('sum')
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
#     add_log('pow')
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке с использованием Pandas DataFrame
def black_list_check(sname):
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if exist:
        return f"{sname}: bad_boy"
    else:
        return f"{sname}: good_boy"

server.register_function(black_list_check, 'black_list_check')

# Проверка нахождения клиента в черном списке с использованием Pandas DataFrame
def black_list_check_fio_birth(surname, name, patronymic, birthdate):
#     add_log('black_list_check_fio_birth')
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')

    # Приведение столбцов к строковому типу для корректного сравнения
    frame['Surname'] = frame['Surname'].astype(str)
    frame['Name'] = frame['Name'].astype(str)
    frame['Patronym'] = frame['Patronym'].astype(str)
    frame['Birth'] = frame['Birth'].astype(str)

    # Проверка на соответствие ФИО и дате рождения
    exist = any((frame['Surname'] == surname) & 
                (frame['Name'] == name) & 
                (frame['Patronym'] == patronymic) & 
                (frame['Birth'] == birthdate)) 

    if exist:
        return f"{surname} {name} {patronymic}, {birthdate}: bad_boy"
    else:
        return f"{surname} {name} {patronymic}, {birthdate}: good_boy"

server.register_function(black_list_check_fio_birth, 'black_list_check_fio_birth')

    

# Бинарная передача данных
def send_back_binary(bin_data):
#     add_log('send_back_binary')
    data = bin_data.data
    return xmlrpc.client.Binary(data)

server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета для изображений RGB (M, N, 3) и монохромных (M, N, 1)
def send_back_inversion(bin_data):
#     add_log('send_back_inversion')
    
    img_arr = pickle.loads(bin_data.data)
    
    # Определяем количество каналов изображения
    if len(img_arr.shape) == 3:  # RGB изображение
        height, width, channels = img_arr.shape
        
        if channels == 3:
            # Обрабатываем RGB изображение
            for i in range(height):
                for j in range(width):
                    img_arr[i, j, 0] = 255 - img_arr[i, j, 0]  # Red
                    img_arr[i, j, 1] = 255 - img_arr[i, j, 1]  # Green
                    img_arr[i, j, 2] = 255 - img_arr[i, j, 2]  # Blue
        
    elif len(img_arr.shape) == 2 or (len(img_arr.shape) == 3 and img_arr.shape[2] == 1):  # Монохромное изображение
        height, width = img_arr.shape[0], img_arr.shape[1]
        
        for i in range(height):
            for j in range(width):
                img_arr[i, j] = 255 - img_arr[i, j]  # Инверсия для монохромного изображения
    
    # Сериализация и возврат результата
    pimg = pickle.dumps(img_arr)
    return xmlrpc.client.Binary(pimg)


server.register_function(send_back_inversion, 'color_inversion')

# Бинаризация изображения по заданному порогу
def binary_threshold(bin_data, threshold):
#     add_log('binary_threshold')
    img_arr = pickle.loads(bin_data.data)

    # Проверка корректности порога
    if threshold < 1 or threshold > 255:
        raise ValueError("Threshold must be in the range 1-255.")

    print('ch: ', len(img_arr.shape))
    # Определяем количество каналов изображения
    if len(img_arr.shape) == 3:  # RGB изображение
        height, width, channels = img_arr.shape

        # Обрабатываем RGB изображение
        for i in range(height):
            for j in range(width):
                # Для каждого канала проверяем и заменяем на 0 или 255
                for c in range(channels):
                    if img_arr[i, j, c] > threshold:
                        img_arr[i, j, c] = 255
                    else:
                        img_arr[i, j, c] = 0

    elif len(img_arr.shape) == 2 or (len(img_arr.shape) == 3 and img_arr.shape[2] == 1):  # Монохромное изображение
        height, width = img_arr.shape[0], img_arr.shape[1]

        for i in range(height):
            for j in range(width):
                if img_arr[i, j] > threshold:
                    img_arr[i, j] = 255
                else:
                    img_arr[i, j] = 0

    # Сериализация и возврат результата
    pimg = pickle.dumps(img_arr)
    return xmlrpc.client.Binary(pimg)

# Регистрация функции на сервере
server.register_function(binary_threshold, 'binary_threshold')


# Разворот изображения относительно вертикали
def vertical_flip(bin_data):
#     add_log('vertical_flip')
    
    img_arr = pickle.loads(bin_data.data)

    # Определяем размерность изображения
    if len(img_arr.shape) == 3:  # RGB изображение
        height, width, channels = img_arr.shape
        
        # Разворот для RGB изображения
        for i in range(height):
            for j in range(width // 2):
                for c in range(channels):
                    # Меняем местами пиксели
                    img_arr[i, j, c], img_arr[i, width - j - 1, c] = img_arr[i, width - j - 1, c], img_arr[i, j, c]

    elif len(img_arr.shape) == 2 or (len(img_arr.shape) == 3 and img_arr.shape[2] == 1):  # Монохромное изображение
        height, width = img_arr.shape[0], img_arr.shape[1]
        
        # Разворот для монохромного изображения
        for i in range(height):
            for j in range(width // 2):
                # Меняем местами пиксели
                img_arr[i, j], img_arr[i, width - j - 1] = img_arr[i, width - j - 1], img_arr[i, j]

    # Сериализация и возврат результата
    pimg = pickle.dumps(img_arr)
    return xmlrpc.client.Binary(pimg)

# Регистрация функции на сервере
server.register_function(vertical_flip, 'vertical_flip')


print ("Listening on port 8008...")
server.serve_forever()


Listening on port 8008...


127.0.0.1 - - [26/Oct/2024 21:52:15] "POST /RPC2 HTTP/1.1" 200 -


ch:  3


127.0.0.1 - - [26/Oct/2024 21:52:30] "POST /RPC2 HTTP/1.1" 200 -


ch:  3


127.0.0.1 - - [26/Oct/2024 21:55:10] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 21:56:46] "POST /RPC2 HTTP/1.1" 200 -


ch:  3


127.0.0.1 - - [26/Oct/2024 21:57:04] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 21:57:12] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 21:57:57] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:00:10] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:00:21] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:01:10] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:01:16] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:01:30] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:01:36] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:03:16] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:03:23] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:03:29] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:03:35] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:03:41] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:03:47] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/202

ch:  3


127.0.0.1 - - [26/Oct/2024 22:06:06] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:06:14] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:06:40] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:07:46] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:08:16] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:08:42] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:09:08] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2024 22:09:34] "POST /RPC2 HTTP/1.1" 200 -
